## Text Classification with TorchText

### Load data with ngrams

In [1]:
import torch
import torchtext
from torchtext.datasets import text_classification
NGRAMS = 2
import os
if not os.path.isdir('./data'):
    os.mkdir('./data')
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./data', ngrams=NGRAMS, vocab=None)
BATCH_SIZE = 16

USE_CUDA = True
if USE_CUDA:
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
        USE_CUDA = False
else:
    device = 'cpu'

120000lines [00:06, 18079.03lines/s]
120000lines [00:13, 8924.20lines/s]
7600lines [00:00, 7959.03lines/s]


### Define the model

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [3]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUM_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUM_CLASS)
model.to(device)

TextSentiment(
  (embedding): EmbeddingBag(1308844, 32, mode=mean)
  (fc): Linear(in_features=32, out_features=4, bias=True)
)

### Functions used to generate batch

In [4]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

### Define functions to train the model and evaluate results.

In [5]:
from torch.utils.data import DataLoader

def train_func(sub_train_):
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                     collate_fn=generate_batch)
    
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text = text.to(device)
        offsets = offsets.to(device)
        cls = cls.to(device)
        
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()
        
    scheduler.step()
    
    return train_loss/len(sub_train_), train_acc/len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size = BATCH_SIZE, collate_fn=generate_batch)
    
    for text, offsets, cls in data:
        text = text.to(device)
        offsets = offsets.to(device)
        cls = cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()
    
    return loss /len(data_), acc / len(data_)

### Split the dataset and run the model

In [6]:
import time
from torch.utils.data.dataset import random_split

N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = random_split(train_dataset, [train_len, len(train_dataset)-train_len])

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)
    
    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60
    
    print(f'EPOCH: {epoch} | time in {mins}m {secs}s')
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc*100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc*100:.1f}%(valid)')    

EPOCH: 0 | time in 0.15m 9s
	Loss: 0.0264(train)	|	Acc: 84.6%(train)
	Loss: 0.0002(valid)	|	Acc: 90.4%(valid)
EPOCH: 1 | time in 0.15m 9s
	Loss: 0.0119(train)	|	Acc: 93.6%(train)
	Loss: 0.0002(valid)	|	Acc: 91.1%(valid)
EPOCH: 2 | time in 0.13333333333333333m 8s
	Loss: 0.0069(train)	|	Acc: 96.4%(train)
	Loss: 0.0001(valid)	|	Acc: 91.5%(valid)
EPOCH: 3 | time in 0.13333333333333333m 8s
	Loss: 0.0039(train)	|	Acc: 98.1%(train)
	Loss: 0.0001(valid)	|	Acc: 90.3%(valid)
EPOCH: 4 | time in 0.13333333333333333m 8s
	Loss: 0.0023(train)	|	Acc: 99.0%(train)
	Loss: 0.0002(valid)	|	Acc: 91.7%(valid)


### Evaluate the model with test dataset

In [7]:
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc*100:.1f}%(test)')

	Loss: 0.0002(test)	|	Acc: 89.5%(test)


### Test on a random news

In [8]:
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

vocab = train_dataset.get_vocab()
model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])

This is a Sports news
